
# Bienvenue au JupyterLab d'Ouranos

Vous êtes sur le serveur JupyterLab d'Ouranos, l'interface de programmation du *Pôle d'Analyse et Visualisation de l'Information Climatique et Scientifique* (PAVICS). PAVICS est une plateforme de recherche spécialisée pour la science du climat, hébergeant à la fois des données:
- Données climatiques en format NetCDF: [THREDDS Data Server](https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/)
- Couches géospatiales: [GeoServer](https://pavics.ouranos.ca/geoserver)

et des services de traitement de données:
- Indicateurs climatiques: [Finch](https://pavics.ouranos.ca/twitcher/ows/proxy/finch/)
- Modélisation hydrologique: [Raven](https://pavics.ouranos.ca/twitcher/ows/proxy/raven/)

Les algorithmes de traitement de données sont accessibles via deux mécanismes:
- les librairies scientifiques installées dans l'environnement de programmation Python du serveur JupyterLab (voir les notebooks dans le répertoire ``pavics-homepage``);
- les requêtes HTML aux services hébergés sur PAVICS, en utilisant le standard *Web Processing Services* (voir l'exemple plus bas).

Si vous accédez à ce JupyterLab à l'aide du compte de démonstration, notez que les ressources de calculs sont limitées, que vos scripts et données sont publics, et qu'ils peuvent être effacés sans préavis. Utilisez uniquement le compte de démonstration pour explorer la plateforme. Vous pouvez faire une demande pour un compte individuel en écrivant à pavics@ouranos.ca.

## Arborescence de votre dossier

```
/notebook_dir/
├── LISMOI.ipynb        # Ce fichier (mode lecture)
├── README.ipynb        # La version en anglais de ce fichier (mode lecture)
├── pavics-homepage     # Les tutoriels de la page d'accueil de PAVICS (mode lecture).
├── tutorial-notebooks  # Tutoriels supplémentaires (mode lecture).
├── public              # Répertoire partagé des autres utilisateurs de la plateforme (mode lecture).
├── mypublic            # Votre répertoire partagé, visible aux autres utilisateurs.
└── writable-workspace  # Votre répertoire personnel privé.
```

## Exemple d'utilisation des services WPS: calculer un indicateur climatique

Pour les utilisateurs qui ne peuvent pas installer un environnement de programmation complet, ou ne veulent pas travailler à partir d'un notebook hébergé sur ce JupyterLab, PAVICS offre une suite de services en ligne accessibles via une interface Web Processing Services (WPS). Cette interface permet d'envoyer une requête à l'un de nos serveurs via une commande HTTP, et recevoir une réponse incluant des liens vers les résultats de la requête. Dans l'exemple qui suit, on utilise le client `birdy` pour faire des appels au serveur Finch afin d'extraire les données d'un point de grille, puis calculer les précipitations maximales annuelles.

In [ ]:
%matplotlib inline
# Connexion au serveur WPS Finch
from birdy import WPSClient
wps = WPSClient('https://pavics.ouranos.ca/twitcher/ows/proxy/finch/')

# Pour inspecter les différents services offerts, faîtes
# wps?

In [ ]:
# Le lien OPENDAP vers les données climatiques
url = "https://pavics.ouranos.ca/thredds/dodsC/birdhouse/"\
      "cmip5/MRI/rcp85/day/atmos/r1i1p1/pr/pr_day_MRI-CGCM3_rcp85_r1i1p1_20060101-20151231.nc"

# Appel WPS vers le service d'extraction d'un point de grille
resp = wps.subset_gridpoint(url, -108, 45, variable='pr')

In [ ]:
# Inspection de la réponse du serveur
reference, meta = resp.get()
print(reference)

Les résultats de l'extraction sont stockés sur le serveur. Vous pouvez réutiliser ce lien comme une entrée vers un autre service de calcul, ou télécharger le fichier localement.

In [ ]:
# Ouvrir une copie locale du fichier
pr, meta = resp.get(asobj=True)
pr

La prochaine étape consiste à calculer la précipitation maximale annuelle à partir de la série extraite et hébergée sur le serveur.

In [ ]:
# Requête pour le calcul de l'indicateur, effectuée sur le serveur à partir des résultats de la requête précédente.
# Notez qu'il est possible de lancer ce genre de calculs sur des fichiers netCDF qui se trouvent sur votre ordinateur personnel ou sur un autre serveur web. Le serveur devra toutefois devoir télécharger (ou streamer) le fichier avant d'effectuer les calculs.

resp_rx1 = wps.rx1day(reference)

In [ ]:
# Téléchargement des résultats
rx1, meta = resp_rx1.get(asobj=True)

In [ ]:
# Visualisation
import xarray as xr
xr.plot.utils.register_pandas_datetime_converter_if_needed()
rx1.rx1day.plot()

## Prochaines étapes

La commande `help(wps)` fait la liste de tous les services offerts par le serveur Finch. La documentation de chaque service est accessible de la même manière (`help(wps.rx1day)`).

Le code présenté plus haut peut être exécuté sur un ordinateur portable peu puissant, car tous les calculs sont effectués sur le serveur. Pour installer les librairies nécessaires, faire:

```
pip install birdhouse-birdy xarray matplotlib
```

Pour plus d'information, consultez la [documentation](https://pavics-sdi.readthedocs.io/en/latest/).

L'équipe d'Ouranos travaille diligemment à l'ajout de nouvelles fonctionnalités et sources de données. Certaines composantes de la plateforme sont plus matures que d'autres, et il est possible que vous rencontriez des difficultés. Pour rapporter un problème, n'hésitez pas à ouvrir un [billet] (https://github.com/Ouranosinc/pavics-sdi/issues).

## Crédits

PAVICS est développé et hébergé par Ouranos, avec le soutien du Centre de Recherche Informatique de Montréal et le Deutsche Klimarechenzentrum (DKRZ), et fait partie de [birdhouse](https://birdhouse.readthedocs.io), une suite de librairies logicielles bâtie par une communauté internationale visant l'accès gratuit à des services et données climatiques. Le développement de la plateforme a été financé par le programme de logiciel de recherche de [CANARIE](https://canarie.ca/fr), le [Centre Canadien de Services Climatiques](https://www.canada.ca/fr/environnement-changement-climatique/services/changements-climatiques/centre-canadien-services-climatiques.html), la [Fondation Canadienne pour l'Innovation](https://www.innovation.ca/fr) et le Plan d'action 2013-2020 sur les changements climatiques du Ministère de l'environnement et de la lutte aux changements climatiques (MDELCC).